In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mt
import seaborn as sns
import numpy as np
import datetime as dt
import math
import sqlite3


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
conn = sqlite3.connect('mimic.db')

In [4]:
admissions = pd.read_sql('select * ,(julianday(date(admittime))-julianday(date(dob)))/365.25 as Age_Admit,(julianday(date(dod))-julianday(date(dob)))/365.25 as Age_Death from admissions, patients where admissions.SUBJECT_ID = patients.SUBJECT_ID',conn)
Diagnoses_ICD = pd.read_sql('select * from diagnoses_icd',conn)
D_ICD_Diagnoses = pd.read_sql('select * from D_ICD_DIAGNOSES',conn)

In [6]:
import datetime
admissions['DOB'] = admissions['DOB'].replace(r'^\s*$', np.nan, regex=True)
admissions['DOD'] = admissions['DOD'].replace(r'^\s*$', np.nan, regex=True)
admissions['ADMITTIME'] = admissions['ADMITTIME'].replace(r'^\s*$', np.nan, regex=True)
admissions['DISCHTIME'] = admissions['DISCHTIME'].replace(r'^\s*$', np.nan, regex=True)
admissions['DOB'] = pd.to_datetime(admissions['DOB'])
admissions['DOD'] = pd.to_datetime(admissions['DOD'])
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'])

In [7]:
# Dropping the duplicated SUBJECT_ID column
data = admissions
data = data.loc[:,~data.columns.duplicated()]

In [8]:
from collections import Counter
# Number of times a patient has been admitted
d ={}
x = data['SUBJECT_ID']
counts = Counter(x)
ID = list(counts.keys())
cnt = list(counts.values())
ID_freq = pd.DataFrame()
ID_freq['SUBJECT_ID'] = ID
ID_freq['ADMISSIONS_CNT'] = cnt
# Merging table
data = data.merge(ID_freq, on='SUBJECT_ID')

In [9]:
# checking if the patient has experienced heart attack
temp = []
value = data['DIAGNOSIS']
x = int(len(data['DIAGNOSIS']))
for i in range(0,x):
    #print(value[i])
    val = value[i]
    #print(val)
    if 'MYOCARDIAL' in val:
        temp.append('1')
    else:
        temp.append('0')
data['HEART_ATTACK'] = temp

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 0 to 58975
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ROW_ID                58976 non-null  object        
 1   SUBJECT_ID            58976 non-null  object        
 2   HADM_ID               58976 non-null  object        
 3   ADMITTIME             58976 non-null  datetime64[ns]
 4   DISCHTIME             58976 non-null  datetime64[ns]
 5   DEATHTIME             58976 non-null  object        
 6   ADMISSION_TYPE        58976 non-null  object        
 7   ADMISSION_LOCATION    58976 non-null  object        
 8   DISCHARGE_LOCATION    58976 non-null  object        
 9   INSURANCE             58976 non-null  object        
 10  LANGUAGE              58976 non-null  object        
 11  RELIGION              58976 non-null  object        
 12  MARITAL_STATUS        58976 non-null  object        
 13  ETHNICITY       

In [11]:
temp = []
for index,row in data.iterrows():
    if(row['HOSPITAL_EXPIRE_FLAG'] == '1'):
        val = row['DIAGNOSIS']
        if 'MYOCARDIAL' in val:
            temp.append(1)
        elif 'CARDIAC' in val:
            temp.append(1)
        elif 'HEART' in val:
            temp.append(1)
        else:
            temp.append(0)
    else:
        temp.append(0)
data['HEART_DEATH'] = temp

In [12]:
# Develop no letter list
list_values = Diagnoses_ICD['ICD9_CODE'].values.tolist()
no_letter_list = []
for i in range(len(list_values)):
    Value = str(list_values[i])
    Letter_stripped_value = Value.lstrip('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    no_letter_list.append(Letter_stripped_value)

    # Develop no number list
list_values = Diagnoses_ICD['ICD9_CODE'].values.tolist()
no_number_list= []
for i in range(len(list_values)):
    Value = str(list_values[i])
    number_stripped_value = Value.lstrip('1234567890')
    if number_stripped_value:
        final_number = number_stripped_value[0]
        no_number_list.append(final_number)
    else:
        no_number_list.append('')      
        
# Correct three digit codes for V codes where decimal is only in first two spots
three_number_list = []
for i in range(len(list_values)):
    if no_number_list[i] == 'V':
        Value = no_letter_list[i] 
        three_digits = Value[0:2]
        three_number_list.append((three_digits))
    else:
        Value = no_letter_list[i]
        three_digits = Value[0:3]
        three_number_list.append((three_digits))

# Add three number list to diagnoses codes
three_number_series = pd.Series(three_number_list)
Diagnoses_ICD['Three Numbers'] = three_number_series # Pull only first three numbers (need to fix for letter codes)

In [13]:
# collect only ages<90
data = data[data['Age_Death'] <= 90]

In [14]:
# Pull Data for all Coronary artery disease patients
Cor_Diagnoses_ICD = Diagnoses_ICD.loc[Diagnoses_ICD['ICD9_CODE'] == '41401'] # All Coronary Artery disease patients
Athero_Diagnoses = pd.DataFrame(Cor_Diagnoses_ICD.SUBJECT_ID)
Athero_Diagnoses = Athero_Diagnoses.drop_duplicates()
Athero_Diagnoses['ATHERO_DIAGNOSIS_FLAG'] = 1


# Merge with demographics file:
data = pd.merge(data, Athero_Diagnoses, on= 'SUBJECT_ID', how = 'outer')

In [15]:
# Add Atherosclerosis diagnosis flags to demographic data
athero_flag = []
for entry in range(len(data)):
    if data['ATHERO_DIAGNOSIS_FLAG'][entry] == 1:
        athero_flag.append(1)
    else:
        athero_flag.append(0)
athero_flag = pd.Series(athero_flag)
data['ATHERO_DIAGNOSIS_FLAG'] = athero_flag

In [16]:
# Info of patients died with heart death
data['HEART_DEATH'].value_counts() 

0.0    19858
1.0      461
Name: HEART_DEATH, dtype: int64

In [17]:
data.shape

(27576, 31)

Data Preprocessing

In [18]:
# Get dummies
data = pd.get_dummies(data, columns=['GENDER','ETHNICITY','MARITAL_STATUS', 'LANGUAGE', 'RELIGION', 'INSURANCE', 'ADMISSION_LOCATION'])

In [19]:
data['HEART_DEATH'].value_counts()

0.0    19858
1.0      461
Name: HEART_DEATH, dtype: int64

In [52]:
data['HEART_ATTACK'].value_counts()

0    19769
1      550
Name: HEART_ATTACK, dtype: int64

In [53]:
data1 =  data.drop(columns = ['HADM_ID','ADMITTIME','DISCHTIME','DEATHTIME','ADMISSION_TYPE','DISCHARGE_LOCATION','EDREGTIME','EDOUTTIME'])

In [54]:
data1 = data1.drop(columns = ['DIAGNOSIS','DOB','DOD','DOD_HOSP','DOD_SSN'])

In [ ]:
data1 = data1.select_dtypes(exclude=['object'])
data1 = data1.fillna(0)


In [61]:
y = data1['HEART_DEATH']
x = data1.drop(columns = ['HEART_DEATH'])
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20, random_state=42)
